In [ ]:
#steps

# Import large Mt5 model
# bring in dataset
# Fine tune
# Evaluate

In [53]:
##Bring in dataset

from datasets import load_dataset

dataset = load_dataset("squad_es",'v1.1.0')


Reusing dataset squad_es (/root/.cache/huggingface/datasets/squad_es/v1.1.0/1.1.0/bcada4f600192451443b95e24f609325705c5185b8aad97bffa8bc3784a867ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87595
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Data Conversion

In [95]:

# Step 1 and 2: Combine context and question, format the answer
from transformers import T5TokenizerFast

# Load the tokenizer
tokenizer = T5TokenizerFast.from_pretrained('google/mt5-base')


def prep_data(train_subset):

    # Create new lists for our inputs and answers
    input_ids = []
    input_attn_msk = []
    answers_ids = []

    for id_value, title_value, context_value, q_value, a_value in zip(train_subset['id'], train_subset['title'], train_subset['context'],train_subset['question'],train_subset['answers']):

        #INPUTS
        # Combine the context and question into a single string
        input_str = context_value + " " + q_value
        #print(input_str)

        #tokenize the inputs
        inputs = tokenizer(input_str, truncation=True, padding='max_length', max_length=512)
        #print(inputs.input_ids)
        input_ids.append(inputs.input_ids)
        input_attn_msk.append(inputs.attention_mask)


        #ANSWERS
        answers = a_value['text']
        answers_tokens = tokenizer(a_value['text'], truncation=True, padding='max_length', max_length=128)
        answers_ids.append(answers_tokens.input_ids)


    result_list = []


    # Iterate over the values from each list
    for input_id, attention_mask, label  in zip(input_ids, input_attn_msk, answers_ids):
        # Create a dictionary with the desired keys
        data_dict = {
            'input_ids': torch.tensor(input_id),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(label)

        }

        # Append the dictionary to the result list
        result_list.append(data_dict)

    return(result_list)


    


loading file https://huggingface.co/google/mt5-base/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/4764ec347af4d2d6286acbe1d9d630ac0afd8554a4c4a64170e0b663fd2e2412.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2
loading file https://huggingface.co/google/mt5-base/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/google/mt5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/mt5-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/0d7d5b3fc19bf58d4b274990c8bcf5e307726bc18d95f40a1436dfb6a0892f85.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
loading file https://huggingface.co/google/mt5-base/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/afba33be693521ccefbde6d03b93b5c517d7108ba31f6c08000ed52c2cea45c9.28bbf90ae7962b1b7211c0ce8b2006f968c82439ec9c47e0847ba63642f9435a

In [100]:


# Start with a subset of your data for testing
train_subset = dataset['train'][:1000]  # Adjust this as needed

model_train_data = prep_data(train_subset)


# Start with a subset of your data for testing
val_subset = dataset['validation'][:100]  # Adjust this as needed

model_val_data = prep_data(val_subset)
          
          

## Data ready test

In [101]:
# replace 'train_dataset' with your dataset variable
sample = model_train_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


In [102]:
# replace 'train_dataset' with your dataset variable
sample = model_val_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


## Fine Tuning

In [104]:
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# Initialize the training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5,
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=validation_data,         # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# Train the model
trainer.train()


loading configuration file https://huggingface.co/google/mt5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ebfd830555547194403d6803baa127970de59b443c04b7a1a60b16a97ed3958.b589da7dac64196f9764abaf2c4c7e507cec8b14b96da3ef270d924f155062de
Model config MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Token

Step,Training Loss
5,52.311000
10,51.443400
15,52.486700
20,51.748900
25,51.464800
30,51.504900
35,51.845000
40,50.478900
45,51.884200
50,50.797500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=80, training_loss=50.781277656555176, metrics={'train_runtime': 67.995, 'train_samples_per_second': 18.825, 'train_steps_per_second': 1.177, 'total_flos': 1534779048591360.0, 'train_loss': 50.781277656555176, 'epoch': 20.0})